In [1]:
from data import load_data
problem = load_data(r"data\100customers\c101.txt", 8, 2, 3)

In [2]:
for customer in problem.customer_list:
    print(customer)

Customer(x=40, y=50, demand=0, time=0.0, service=0.0)
Customer(x=45, y=68, demand=10, time=11.4, service=0.075)
Customer(x=-45, y=-70, demand=30, time=10.3125, service=0.075)
Customer(x=42, y=66, demand=10, time=0.8125, service=0.075)
Customer(x=42, y=68, demand=10, time=9.0875, service=0.075)
Customer(x=42, y=65, demand=10, time=0.1875, service=0.075)
Customer(x=40, y=69, demand=20, time=7.7625, service=0.075)
Customer(x=40, y=66, demand=20, time=2.125, service=0.075)
Customer(x=38, y=68, demand=20, time=3.1875, service=0.075)


In [3]:
from problem import Problem
from population import Individual
def extract_routes(chromosome, problem: Problem):
    truck_routes = []
    drone_routes = []
    tmp_truck_route = []
    tmp_drone_route = []

    k = problem.number_customer + 1
    d = problem.number_customer + problem.number_of_trucks

    for i in range(len(chromosome[0])):
        if chromosome[0][i] >= k and chromosome[0][i] < d:
            truck_routes.append(tmp_truck_route)
            tmp_truck_route = []
            continue
        elif (
            chromosome[0][i] >= d
            and chromosome[0][i] < d + problem.number_of_drones - 1
        ):
            drone_routes.append(tmp_drone_route)
            tmp_drone_route = []
            continue

        if chromosome[1][i] == 0:
            tmp_truck_route.append(chromosome[0][i])
        elif chromosome[1][i] == 1:
            tmp_drone_route.append(chromosome[0][i])

    truck_routes.append(tmp_truck_route)
    drone_routes.append(tmp_drone_route)
    return truck_routes, drone_routes

def repair_capacity(chromosome, problem: Problem):
    for i in range(len(chromosome[0])):
        if chromosome[1][i] == 1 and chromosome[0][i] <= problem.number_customer:
            if problem.customer_list[chromosome[0][i]].quantity > problem.drone_capacity:
                chromosome[1][i] = 0
    return chromosome

def repair_distance(chromosome, problem: Problem):
    max_distance = (
        (problem.drone_energy / (problem.energy_consumption_rate * problem.weight_of_drone))
        - problem.land_time
        - problem.launch_time
    ) * problem.speed_of_drone

    for i in range(len(chromosome[0])):
        customer = chromosome[0][i]
        if chromosome[1][i] == 1 and customer <= problem.number_customer:
            nearest_dist = float("inf")
            for j in range(len(chromosome[0])):
                if i == j:
                    continue
                other = chromosome[0][j]
                if other <= problem.number_customer:
                    dist = problem.distance_matrix_drone[customer][other]
                    if dist < nearest_dist:
                        nearest_dist = dist

            if nearest_dist == float("inf") or nearest_dist > max_distance:
                chromosome[1][i] = 0

    return chromosome


def find_nearest_customer(customer_drone_list, customer_truck_list, distance_drone_matrix):
    nearest_dict = {}

    for customer in customer_drone_list:
        distances = [
            (other, distance_drone_matrix[customer][other])
            for other in customer_truck_list if other != customer
        ]
        distances.append((0, distance_drone_matrix[0][customer]))
        distances.sort(key=lambda x: x[1])
        nearest_dict[customer] = [other for other, _ in distances]

    return nearest_dict


def update_nearest_dict(nearest_dict, customer_remove, distance_drone_matrix):
    if customer_remove in nearest_dict:
        del nearest_dict[customer_remove]

    for key, value in nearest_dict.items():
        dist_remove = distance_drone_matrix[key][customer_remove]

        inserted = False
        new_list = []
        for other in value:
            dist_other = distance_drone_matrix[key][other]
            if not inserted and dist_remove < dist_other:
                new_list.append(customer_remove)
                inserted = True
            new_list.append(other)
        if not inserted:
            new_list.append(customer_remove)

        nearest_dict[key] = new_list

    return nearest_dict



In [ ]:
from problem import Truck_Solution, Drone_Solution, Drone_Trip
from copy import deepcopy
import numpy as np
def init_truck_solution(route, problem: Problem):
    num_point = len(route)
    if num_point == 0:
        return Truck_Solution([], [], [], [], [])
    assigned_customer = deepcopy(route)
    recived_truck = np.zeros(num_point)
    recived_drone = np.zeros(num_point)
    arrive_time = np.zeros(num_point)
    leave_time = np.zeros(num_point)
    arrive_time[0] = problem.distance_matrix_truck[0][route[0]]/problem.speed_of_truck
    recived_truck[0] = problem.customer_list[route[0]].quantity
    recived_drone[0] = 0
    leave_time[0] = max(arrive_time[0], problem.customer_list[route[0]].arrive_time) + problem.customer_list[route[0]].service_time
    for i in range(1, num_point):
        arrive_time[i] = leave_time[i-1] + problem.distance_matrix_truck[route[i-1]][route[i]]/problem.speed_of_truck
        recived_truck[i] = problem.customer_list[route[i]].quantity
        recived_drone[i] = 0
        leave_time[i] = max(arrive_time[i], problem.customer_list[route[i]].arrive_time) + problem.customer_list[route[i]].service_time
    truck_solution = Truck_Solution(assigned_customer, recived_truck, recived_drone, arrive_time, leave_time)
    return truck_solution


def update_truck_solution(truck_solution: Truck_Solution, problem:Problem, customer, add_quantity, leave_arrive, index_insert = None):
    if customer in  truck_solution.assigned_customers:
        idx = truck_solution.assigned_customers.index(customer)
        truck_solution.recived_drone[idx] += add_quantity
        truck_solution.leave_time[idx] = max(truck_solution.leave_time[idx], leave_arrive)
        if truck_solution.leave_time[idx] == leave_arrive:
            route = truck_solution.assigned_customers
            for i in range(idx+1, len(truck_solution.assigned_customers)):
                truck_solution.arrive_time[i] = truck_solution.leave_time[i-1] + problem.distance_matrix_truck[route[i-1]][route[i]]/problem.speed_of_truck
                truck_solution.leave_time[i] = max(truck_solution.arrive_time[i], problem.customer_list[route[i]].arrive_time) + problem.customer_list[route[i]].service_time
        return truck_solution


    truck_solution.assigned_customers.insert(index_insert, customer)
    truck_solution.recived_truck = np.insert(truck_solution.recived_truck, index_insert, 0)
    truck_solution.recived_drone = np.insert(truck_solution.recived_drone, index_insert, add_quantity)
    truck_solution.arrive_time = np.insert(truck_solution.arrive_time, index_insert, 0)
    truck_solution.leave_time = np.insert(truck_solution.leave_time, index_insert, 0)

    route = truck_solution.assigned_customers
    n = len(route)

    if index_insert == 0:
        truck_solution.arrive_time[0] = problem.distance_matrix_truck[0][route[0]] / problem.speed_of_truck
    else:
        truck_solution.arrive_time[index_insert] = truck_solution.leave_time[index_insert-1] + problem.distance_matrix_truck[route[index_insert-1]][route[index_insert]] / problem.speed_of_truck

    truck_solution.leave_time[index_insert] = max(max(truck_solution.arrive_time[index_insert],
                                                 problem.customer_list[route[index_insert]].arrive_time) + problem.customer_list[route[index_insert]].service_time, leave_arrive)

    for i in range(index_insert+1, n):
        truck_solution.arrive_time[i] = truck_solution.leave_time[i-1] + problem.distance_matrix_truck[route[i-1]][route[i]] / problem.speed_of_truck
        truck_solution.leave_time[i] = max(truck_solution.arrive_time[i],
                                           problem.customer_list[route[i]].arrive_time) + \
                                       problem.customer_list[route[i]].service_time

    return truck_solution



In [ ]:
def find_solution(truck_routes, drone_trips, problem: Problem):
    truck_solutions = []
    for route in truck_routes:
        truck_solutions.append(init_truck_solution(route, problem))

    drone_solutions = []

    for trip in drone_trips:
        if len(trip) == 0:
            drone_solutions.append(Drone_Solution(0, None))
            continue
        customer_on_trip = [(customer, problem.customer_list[customer].arrive_time) for customer in trip]
        customer_on_trip.sort(key= lambda x: x[1])

        drone_trip_customer_assigned = []
        recived_drone = []
        arrive_time = []
        leave_time = []

        
        drone_trip = Drone_Trip()

    

        

In [6]:
chromosome = [[1, 10, 3, 7, 4, 9, 5, 6, 11, 8, 2], [0,  0, 0, 1, 0, 0, 1, 0,  0, 1, 1]]
truck_routes, drone_trips = extract_routes(chromosome, problem)

print("assigned_truck:", truck_routes)
print("assgined_drone", drone_trips)

nearest_dict = find_nearest_customer([7,5, 8], [1,2,3,4,6], problem.distance_matrix_drone)
print(nearest_dict)
nearest_dict = update_nearest_dict(nearest_dict, 5, problem.distance_matrix_drone)
print(nearest_dict)

assigned_truck: [[1, 3, 4], [6]]
assgined_drone [[], [7, 5], [8, 2]]
{7: [3, 4, 6, 1, 0, 2], 5: [3, 4, 1, 6, 0, 2], 8: [6, 4, 3, 1, 0, 2]}
{7: [3, 5, 4, 6, 1, 0, 2], 8: [6, 4, 3, 5, 1, 0, 2]}


In [7]:
chromosome = repair_capacity(chromosome, problem)
print(chromosome)
chromosome = repair_distance(chromosome, problem)
print(chromosome)

[[1, 10, 3, 7, 4, 9, 5, 6, 11, 8, 2], [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1]]
[[1, 10, 3, 7, 4, 9, 5, 6, 11, 8, 2], [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]]


In [8]:
truck_solution = init_truck_solution([1,3,4], problem)
print(truck_solution.assigned_customers)
print(truck_solution.recived_truck)
print(truck_solution.recived_drone)
print(truck_solution.arrive_time)
print(truck_solution.leave_time)

[1, 3, 4]
[10. 10. 10.]
[0. 0. 0.]
[ 0.575 11.6   11.725]
[11.475 11.675 11.8  ]


In [9]:
truck_solution = init_truck_solution([1,3,4], problem)
print(truck_solution.assigned_customers)
print(truck_solution.recived_truck)
print(truck_solution.recived_drone)
print(truck_solution.arrive_time)
print(truck_solution.leave_time)

print("************************")
truck_solution = update_truck_solution(truck_solution, problem, 2, 10, 15, 0)
print(truck_solution.assigned_customers)
print(truck_solution.recived_truck)
print(truck_solution.recived_drone)
print(truck_solution.arrive_time)
print(truck_solution.leave_time)
print("******************")
truck_solution = update_truck_solution(truck_solution, problem, 6, 10, 15, 2)
print(truck_solution.assigned_customers)
print(truck_solution.recived_truck)
print(truck_solution.recived_drone)
print(truck_solution.arrive_time)
print(truck_solution.leave_time)

[1, 3, 4]
[10. 10. 10.]
[0. 0. 0.]
[ 0.575 11.6   11.725]
[11.475 11.675 11.8  ]
************************
[2, 1, 3, 4]
[ 0. 10. 10. 10.]
[10.  0.  0.  0.]
[ 5.125 20.7   20.9   21.025]
[15.    20.775 20.975 21.1  ]
******************
[2, 1, 6, 3, 4]
[ 0. 10.  0. 10. 10.]
[10.  0. 10.  0.  0.]
[ 5.125 20.7   20.925 21.125 21.25 ]
[15.    20.775 21.    21.2   21.325]
